# SQLAlchemy 1.4 / 2.0 Tutorial

## Establishing Connectivity - the Engine

In [14]:
import os.path
import yaml
from sqlalchemy import create_engine, text

config_path = os.path.expanduser('~\\.sqlalchemy\\config.yaml')

with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

db_path = (
    f"postgresql://{config['username']}:{config['password']}"
    f"@{config['host']}:{config['port']}/{config['db_name']}"
)
engine = create_engine(db_path, echo=True, future=True)

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())


2021-11-18 13:58:48,954 INFO sqlalchemy.engine.Engine select version()
2021-11-18 13:58:48,954 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-18 13:58:48,955 INFO sqlalchemy.engine.Engine select current_schema()
2021-11-18 13:58:48,956 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-18 13:58:48,957 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-11-18 13:58:48,958 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-18 13:58:48,959 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-18 13:58:48,960 INFO sqlalchemy.engine.Engine select 'hello world'
2021-11-18 13:58:48,961 INFO sqlalchemy.engine.Engine [generated in 0.00126s] {}
[('hello world',)]
2021-11-18 13:58:48,963 INFO sqlalchemy.engine.Engine ROLLBACK
